# Car Colour Detection & Counting

This Jupyter notebook demonstrates a Streamlit app that:
- Detects **cars** and **people** in images (using YOLOv8).
- For each car:
  - **Red rectangle** if it's **blue**.
  - **Blue rectangle** otherwise.
- Shows total counts of cars and people.


In [ ]:
!pip install ultralytics opencv-python numpy streamlit Pillow

In [ ]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image

st.set_page_config(page_title="Car Colour Detection & Counting", layout="wide")

@st.cache_resource
def load_model():
    return YOLO("yolov8n.pt")

model = load_model()

def is_blue_car(crop_bgr, blue_thresh=0.20):
    if crop_bgr is None or crop_bgr.size == 0:
        return False
    hsv = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    mask_valid = (s > 60) & (v > 50)
    h_valid = h[mask_valid]
    if h_valid.size == 0:
        return False
    blue_mask = (h_valid >= 90) & (h_valid <= 135)
    blue_ratio = np.count_nonzero(blue_mask) / float(h_valid.size)
    return blue_ratio >= blue_thresh

def draw_results(image_bgr, boxes, classes, names):
    out = image_bgr.copy()
    car_count = 0
    people_count = 0
    for box, cls in zip(boxes, classes):
        cls_name = names.get(int(cls), str(int(cls)))
        x1, y1, x2, y2 = [int(v) for v in box]
        if cls_name == "person":
            people_count += 1
        if cls_name == "car":
            car_count += 1
            crop = out[max(0,y1):max(0,y2), max(0,x1):max(0,x2)]
            blue = is_blue_car(crop)
            rect_color = (0,0,255) if blue else (255,0,0)
            label = "BLUE car" if blue else "car"
            cv2.rectangle(out, (x1,y1), (x2,y2), rect_color, 2)
            cv2.putText(out, label, (x1, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, rect_color, 2, cv2.LINE_AA)
    cv2.putText(out, f"Cars: {car_count}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(out, f"People: {people_count}", (10,65), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    return out, car_count, people_count

def process_image(image_bgr):
    res = model(image_bgr, verbose=False)[0]
    boxes = res.boxes.xyxy.cpu().numpy() if res.boxes is not None else np.empty((0,4))
    classes = res.boxes.cls.cpu().numpy().astype(int) if res.boxes is not None else np.empty((0,))
    names = res.names
    return draw_results(image_bgr, boxes, classes, names)

st.title("Car Colour Detection & Counting")
st.caption("Red rectangle for BLUE cars | Blue rectangle for other cars | Counts cars and people")

tabs = st.tabs(["Upload Image", "Webcam Preview (optional)"])

with tabs[0]:
    file = st.file_uploader("Upload traffic image", type=["jpg","jpeg","png"])
    if file is not None:
        image = Image.open(file).convert("RGB")
        st.image(image, caption="Input preview", use_column_width=True)
        img_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        if st.button("Run Detection on Image"):
            out, cars, people = process_image(img_bgr)
            st.image(cv2.cvtColor(out, cv2.COLOR_BGR2RGB), caption=f"Cars: {cars} | People: {people}", use_column_width=True)

with tabs[1]:
    run = st.checkbox("Start webcam")
    frame_window = st.empty()
    if run:
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            st.error("Could not open webcam.")
        else:
            while run:
                ok, frame = cap.read()
                if not ok:
                    st.error("Webcam frame unavailable.")
                    break
                out, cars, people = process_image(frame)
                frame_window.image(cv2.cvtColor(out, cv2.COLOR_BGR2RGB), channels="RGB", use_column_width=True)
                run = st.checkbox("Start webcam", value=True)
            cap.release()
            frame_window.empty()


# Running the App

After running the cells above, launch the Streamlit app with:

```bash
!streamlit run app.py
```

Then click the link shown to open it in your browser.
